In [1]:
import numpy as np
import torch

# dataset

def create_dataset_RNN_addition():
    
    seed = 0
    np.random.seed(seed)
    
    seq_length = 5
    max_num = 2**seq_length
    X, Y = np.meshgrid(np.arange(max_num), np.arange(max_num))
    x = np.transpose(np.array([X.reshape(-1,), Y.reshape(-1,)]))
    rs = []
    routs = []

    y = x[:,[0]] + x[:,[1]]

    for i in range(seq_length):
        x, r = np.divmod(x, 2)
        rs.append(r)

    for i in range(seq_length+1):
        y, rout = np.divmod(y, 2)
        routs.append(rout)


    data = np.transpose(np.array(rs), (1,0,2))

    inputs = np.concatenate([data, np.zeros((max_num**2, 1, 2))], axis=1)
    labels = np.transpose(np.array(routs), (1,0,2))


    carrybit = (inputs[:,:,0] + inputs[:,:,1] - labels[:,:,0]) % 2
    currentbit = inputs[:,:-1,0] + inputs[:,:-1,1]

    train_frac = 0.8
    num = inputs.shape[0]
    train_num = int(train_frac*num)
    test_num = num - train_num

    train_id = np.random.choice(num, train_num, replace=False)
    test_id = np.array(list(set(np.arange(num,)) - set(train_id)))

    inputs_train = inputs[train_id]
    labels_train = labels[train_id]

    inputs_test = inputs[test_id]
    labels_test = labels[test_id]
    
    def mat2str(mat):
        temp = mat.astype(int).astype(str)
        num = temp.shape[0]
        seq_len = temp.shape[1]
        strs = []
        for i in range(num):
            bit_str = ''
            for j in range(seq_len):
                bit_str += temp[i][j]
            strs.append(bit_str)
        return strs
    
    data_train = np.transpose(np.array([mat2str(inputs_train[:,:,0]), mat2str(inputs_train[:,:,1]), mat2str(labels_train[:,:,0])]))
    data_test = np.transpose(np.array([mat2str(inputs_test[:,:,0]), mat2str(inputs_test[:,:,1]), mat2str(labels_test[:,:,0])]))
    
    np.savetxt('./data_train.txt', data_train, fmt='%s')
    np.savetxt('./data_test.txt', data_test, fmt='%s')
    
create_dataset_RNN_addition()